In [1]:
%load_ext autoreload
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from dbmaster import MasterParams, MasterPersist
from dbbase import MusicDBIDModVal, MusicDBDir, MusicDBData
from dbnote import DownloadRecord, KnownRecord, MergeSearchArtist
from utils import FileIO, DirInfo, FileInfo, getFlatList, Timestat, TermTime, TermTimeTS, getTT
from pandas import Series, DataFrame, concat, Timestamp
from pandb import PanDBIO
from musicdb.applemusic import MusicDBParams, RawWebData, MusicDBIO, MusicDBID
from os import getpid

mv = MusicDBIDModVal()
io = FileIO()
mpar = MusicDBParams()
dbio = MusicDBIO()
webio = RawWebData()
db = mpar.db

Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]


/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


In [3]:
searchArtistRecord = DownloadRecord(db=db, name="SearchArtist", rTypes=["Index", "Data"])
downloadArtistRecord = DownloadRecord(db=db, name="DownloadArtist", rTypes=["Index"])
downloadAlbumRecord = DownloadRecord(db=db, name="DownloadAlbum", rTypes=["Index"])
knownAlbumRecord = KnownRecord(db=db, name="KnownAlbum", rTypes=["Index", "Data"])
knownPlaylistRecord = KnownRecord(db=db, name="KnownPlaylist", rTypes=["Index", "Data"])
knownCuratorRecord = KnownRecord(db=db, name="KnownCurator", rTypes=["Index", "Data"])
knownStationRecord = KnownRecord(db=db, name="KnownStation", rTypes=["Index", "Data"])
knownVideoRecord = KnownRecord(db=db, name="KnownVideo", rTypes=["Index", "Data"])
knownPostRecord = KnownRecord(db=db, name="KnownPost", rTypes=["Index", "Data"])
knownArtists = dbio.rdio.getSummaryNameData

Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=AppleMusic, name=SearchArtist, rTypes=['Index', 'Data'])
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=AppleMusic, name=DownloadArtist, rTypes=['Index'])
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=AppleMusic, name=DownloadAlbum, rTypes=['Index'])
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=AppleMusic, name=KnownAlbum)
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=AppleMusic, name=KnownPlaylist)
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=AppleMusic, name=KnownCurator)
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=AppleMusic, name=KnownStation)
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=AppleMusic, name=KnownVideo)
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
DownloadRecord(db=AppleMusic, name=KnownPost)


In [ ]:
if False:
    from utils import FileInfo
    import shutil
    db = dbio.params.db
    dbdir = dbio.params.dir
    nameMap = {"MasterArtists": "SearchArtist", "LocalArtists": "DownloadArtist", "LocalAlbums": "DownloadAlbum"}
    dinfo = DirInfo(f"/Users/tgadfort/anaconda3/envs/post/pandb/musicdb/{db}")
    
    cpMap = {}
    for origname, newname in nameMap.items():
        cpMap[f"{dbdir}SearchedFor{origname}"] = f"{db}-{newname}-Index"
        cpMap[f"{dbdir}SearchedFor{origname}Data"] = f"{db}-{newname}-Data"
        cpMap[f"{dbdir}SearchedFor{origname}Errors"] = f"{db}-{newname}-Error"
    assert dinfo.exists(), f"{dinfo} does not exist!"
    for srcFile, dstFile in cpMap.items():
        srcFile = dinfo.join(f"{srcFile}.p")
        dstFile = dinfo.join(f"{dstFile}.p")
        if srcFile.exists():
            srcFile.mvFile(dstFile)

In [ ]:
##########################################################################################
# Show Summary
##########################################################################################
print(f"{db} Search Results (PID={getpid()})".format(db))
searchArtistRecord.info()
downloadArtistRecord.info()
downloadAlbumRecord.info()
knownAlbumRecord.info()
knownPlaylistRecord.info()
knownCuratorRecord.info()
knownStationRecord.info()
knownVideoRecord.info()
knownPostRecord.info()
print(f"  {'KnownArtist Data': <20}: {knownArtists().shape[0]}")

# Search For New Artists

In [4]:
######################################################################################################
# Explode Recent Data
######################################################################################################
def getNewData(searchArtistRecord):
    newData = searchArtistRecord.getData()
    if len(newData) == 0:
        return DataFrame()
    newData = concat([DataFrame(s["Artist"]).T for s in newData.values()])
    newData.index = newData.index.astype(str)
    newData = newData[~newData.index.duplicated()]
    return newData

def mergeNewData(searchArtistRecord):
    searchArtistRecord.load(verbose=False)
    newData = searchArtistRecord.getData()
    if len(newData) == 0:
        return
    for idType in ['Album', 'Playlist', 'Curator', 'Station', 'Video', 'Post']:
        keyData = {key: record.get(idType, {}) for key, record in newData.items()}
        keyData = {key: value for s in keyData.values() for key, value in s.items()}
        if len(keyData) == 0:
            continue
        exec(f"known{idType}Record.load(verbose=False)")
        exec(f"known{idType}Record.mergeData(keyData)")
        exec(f"known{idType}Record.save()")
        
    newData = getNewData(searchArtistRecord)
    msr = MergeSearchArtist(db)
    msr.mergeLocal(searchArtistRecord, newData, test=False)

In [8]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
webio = RawWebData(debug=False)
knownNames = PanDBIO().getUniqueUnmatchedArtistNames(db)
mergeNewData(searchArtistRecord)
searchedNames = Series(searchArtistRecord.getIndex())
artistNamesToGet = knownNames[~knownNames.map(searchArtistRecord.isKnown)]

print(f"# {db} Search Results (PID={getpid()})")
print(f"#   Available Names:     {knownNames.shape[0]}")
print(f"#   Known Artist Names:  {searchedNames.shape[0]}")
print(f"#   Artist Names To Get: {artistNamesToGet.shape[0]}")

del searchedNames
del knownNames

/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
*****************************************************************************************************************************
*                                                     Saving KnownAlbum                                                     *
*****************************************************************************************************************************
  Saving Data [2046624] ... Done
****************************************************************************************************************************
*                                                   Saving KnownPlaylist                                                   *
****************************************************************************************************************************
  Saving Data [36949] ... Done
************************

/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


  ==> Created copy of local SearchArtist data with (1661347, 3) shape
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]


/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


Found 18500 Searches
Found 144872 Results
Found 0 Duplicate Results
Found 1661347 Previously Known Results
Found 55069 New Results
  ==> 7.83 All Results / Search
  ==> 2.98 New Results / Search
Saving 1716416 Data ... Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]


/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


Done
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
*****************************************************************************************************************************
*                                                 Initializing SearchArtist                                                 *
*****************************************************************************************************************************
  Initializing Data   ... Done
# AppleMusic Search Results (PID=65638)
#   Available Names:     1012663
#   Known Artist Names:  333932
#   Artist Names To Get: 678644


In [7]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def isError(artistName, nErrors, sleeptime, error=None):
    searchArtistRecord.setError(index=artistName)    
    print(f"Search Error ==> {artistName}: {error}")
    nErrors.append(artistName)
    webio.sleep(sleeptime)
    

n = 0
maxN = 25000000
nErrors = []
searchArtistRecord.load(verbose=False)
for i, (idx, artistName) in enumerate(artistNamesToGet.items()):
    if searchArtistRecord.isKnown(artistName):
        continue

    if len(nErrors) >= 5:
        print("Stopping due to 5 consecutive errors")
        break

    try:
        response = webio.getArtistSearchData(artistName=artistName)
    except Exception as error:
        isError(artistName, nErrors, 10, error)
        continue
    
    if not isinstance(response, dict):
        isError(artistName, nErrors, 7.5, "NotDict")
        continue

    nErrors = []
    searchArtistRecord.setData(index=artistName, data=response)
    webio.sleep(5.0)
    n += 1
        
    if n % 25 == 0:
        ts.update(n=n)
        searchArtistRecord.save()
        webio.wait(5.0)
        if tt.isFinished():
            break
    
    if n >= maxN:
        print(f"Breaking after {maxN} downloads...")
        break

ts.stop()
searchArtistRecord.save()

 [Getting AppleMusic ArtistIDs] | StartTime 2023-11-24 10:59:52
   ====> Terminate Time Set To 2023-11-24 19:00:00 <====
   ====> Will Terminate Process 8 Hour and 7 Second From Now
Searching For Akkai                                             {'Artist': 19, 'Album': 12}
Searching For Yung Sheikh                                       {'Artist': 13, 'Album': 9}
Searching For Pathetic Son                                      {'Artist': 1, 'Album': 13}
Searching For Nick Rossi                                        {'Artist': 3, 'Album': 9}
Searching For Salvatore di Giacomo                              {'Artist': 20, 'Album': 11}
Searching For Cir:cle                                           {'Artist': 14, 'Album': 9}
Searching For Gustavo Dominguez                                 {'Artist': 13, 'Album': 14}
Searching For Minor G                                           {'Artist': 31, 'Album': 12}
Searching For Dr. Thulz                                         {'Artist': 9, 'Album': 

Waiting:   0%|          | 0/50 [00:00<?, ?it/s]

Searching For School Daze                                       {'Artist': 15, 'Album': 16}
Searching For Hard & Pure                                       {'Artist': 16, 'Album': 15}
Searching For Vegar Dahl                                        {'Artist': 2, 'Album': 11}
Searching For P.H.A.T.T.                                        {'Artist': 32, 'Album': 17}
Searching For Emre Arisev                                       {'Artist': 5, 'Album': 14}
Searching For Riva's Groove                                     {'Artist': 10, 'Album': 20}
Searching For Trey Gordon                                       {'Artist': 2, 'Album': 9}
Searching For Ruven Medici                                      {'Artist': 7, 'Album': 15}
Searching For Arctic Domination                                 {'Artist': 2, 'Album': 8}
Searching For Atari Alan                                        {'Artist': 5, 'Album': 14}
Searching For Tong & Spoon                                      {'Artist': 8, 'Album': 1

Waiting:   0%|          | 0/50 [00:00<?, ?it/s]

Searching For Acid on Sphere                                    {'Artist': 5, 'Album': 13}
Searching For Soyana                                            {'Artist': 5, 'Album': 13}
Searching For Massimo Mantovani                                 {'Artist': 3, 'Album': 10}
Searching For Destiny Navaira                                   {'Artist': 6, 'Album': 11}
Searching For Ask Carol                                         {'Artist': 1, 'Album': 10}
Searching For Rene Matamoros                                    {'Artist': 2, 'Album': 8}
Searching For Priyanka NK                                       {'Artist': 29, 'Album': 15}
Searching For The Rex-Factor                                    {'Artist': 4, 'Album': 16}
Searching For L-Smooth Mensah                                   {'Artist': 7, 'Album': 9}
Searching For The Dead Reds                                     {'Artist': 19, 'Album': 11}
Searching For Moshe Siegel                                      {'Artist': 5, 'Album': 11}

...

Searching For Morfiouz                                          {'Artist': 1, 'Album': 7}
Searching For Leonardo Zaccaria                                 {'Artist': 3, 'Album': 13}
Searching For Monte Cristo                                      {'Artist': 26, 'Album': 14}
Searching For Rayos                                             {'Artist': 22, 'Album': 19}
Searching For Chill Armonia Studio                              {'Artist': 5, 'Album': 3}
Searching For Ode to the Marionette                             {'Artist': 2, 'Album': 3}
Searching For Mesotec                                           {'Artist': 3, 'Album': 10}
Searching For Sara Dziri                                        {'Artist': 1, 'Album': 3}
Searching For Yuuki Ozaki                                       {'Artist': 9, 'Album': 8}
Searching For Ricky Rick                                        {'Artist': 24, 'Album': 16}
Searching For Package                                           {'Artist': 26, 'Album': 12}


Waiting:   0%|          | 0/50 [00:00<?, ?it/s]

Searching For Fevertree                                         {'Artist': 8, 'Album': 10}
Searching For Alberto Benita                                    {'Artist': 5, 'Album': 3}
Searching For Common Man                                        {'Artist': 24, 'Album': 14}
Searching For Sukanta Adhikari                                  {'Artist': 1, 'Album': 1}
Searching For Suganya Migael                                    {'Artist': 1, 'Album': 2}
Searching For Deca Ydf                                          {'Artist': 3, 'Album': 9}
Searching For Out of Favor Boys                                 {'Artist': 1, 'Album': 5}
Searching For Mildred Clark & The Melodyaires                   {'Artist': 1, 'Album': 7}
Searching For Infantry                                          {'Artist': 24, 'Album': 16}
Searching For Tilo Escobar                                      {'Artist': 1, 'Album': 6}
Searching For Anita Dahlan                                      {'Artist': 1, 'Album': 6}
Searc

Waiting:   0%|          | 0/50 [00:00<?, ?it/s]

   ====> Terminate Time [2023-11-24 19:00:00] Is Reached <====
 [Getting AppleMusic ArtistIDs] | RunTime 8 Hour and 28 Second.  2023-11-24 19:00:20
*****************************************************************************************************************************
*                                                    Saving SearchArtist                                                    *
*****************************************************************************************************************************
  Saving Index [333932] ... Done
  Saving Data [18500] ... Done
  Saving Error [88] ... Done


In [ ]:
searchArtistRecord.save()

In [ ]:
from utils import getHTML
bsdata = getHTML(response.data)

def getLinkData(idtype: str, refs: list) -> 'dict':
    def getBasicData(ref):
        url = ref.get('href')
        txt = ref.text.replace("\n", "").strip()
        name = txt if len(txt) > 0 else None
        return url, name
    
    linkdata = {}
    for ref in refs:
        url, name = getBasicData(ref)
        func = getattr(webio.mdbid, f"get{idtype}ID") if hasattr(webio.mdbid, f"get{idtype}ID") else None
        dbid = func(url) if callable(func) else None
        if all([isinstance(obj, str) for obj in [url, dbid, name]]):
            linkdata[dbid] = {"URL": url, "Name": name}
    return linkdata

linktypes = {"artist": "Artist", "album": "Album", "playlist": "Playlist", "curator": "Curator",
             "station": "Station", "music-video": "Video", "post": "Post"}
links = bsdata.findAll("a")
retval = {}
for refstub, idtype in linktypes.items():
    refs = [ref for ref in links if f"/{refstub}/" in ref.get('href')]
    retval[idtype] = getLinkData(idtype, refs)

status = {idtype: len(retval[idtype]) for idtype in ["Artist", "Album"]}
print(status)

In [ ]:
bsdata

In [ ]:
searchArtistRecord.load()

In [ ]:
dels = []
above = 1000000000
for i,(k,v) in enumerate(searchArtistRecord.recordData['Index'].items()):
    if k == "Confront":
        above = i
    if i >= above:
        dels.append(k)

In [ ]:
for name in dels:
    del searchArtistRecord.recordData['Index'][name]
    del searchArtistRecord.recordData['Data'][name]

In [ ]:
searchArtistRecord.save()

In [9]:
msa = MergeSearchArtist(dbio.db)
msa.copyFromLocal(force=True)

****************************************************************************************************************************
*                                             MergeSearchArtist(db=AppleMusic)                                             *
****************************************************************************************************************************
New Local Shape:  (1716416, 3)
Old Global Shape: (890639, 3)
New Global Shape: (1716416, 3)


# Download Artist Data

In [ ]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
webio = RawWebData(debug=False)

In [ ]:
searchArtistRecord.load()

In [ ]:
df

In [ ]:
artistNames            = searchArtists()
artistNames['IndexModVal'] = artistNames.index.map(mv.getModVal)
localArtistsDict       = localArtists.get()
localArtistsErrorsDict = localArtistsErrors.get()
availableNames         = artistNames[~artistNames.index.isin(localArtistsDict.keys())]
availableNames         = availableNames[~availableNames.index.isin(localArtistsErrorsDict.keys())]
artistNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby(["IndexModVal"])})


print("# {0} Search Results".format(db))
print("#   Available Names:      {0}".format(len(artistNames)))
print("#   Known Artist Names:   {0}".format(len(localArtistsDict)))
print("#   Artist Names To Get:  {0}".format(len(availableNames)))

del availableNames
del localArtistsErrorsDict
del localArtistsDict
del artistNames

#   Artist Names To Get:  3051

In [ ]:
def saveArtistData(db, localArtistsDict, localArtistsDictErrors):
    print("="*150)
    print(f"Saving {len(localArtistsDict)} {db} Artists Data")
    localArtists.save(data=localArtistsDict)
    print(f"Saving {len(localArtistsDictErrors)} {db} Artists Data Errors")
    localArtistsErrors.save(data=localArtistsDictErrors)
    print("="*150)

In [ ]:
ts = Timestat("Getting {0} ArtistIDs".format(db))
tt = getTT(skipEOD=False)

n    = 0
maxN = 2500000
localArtistsDict = localArtists.get()
localArtistsDictErrors = localArtistsErrors.get()
stop = False
nErrors = []
for groupModVal,modValData in artistNamesToGet.iteritems():
    if stop is True:
        break
    for j,(artistID,row) in enumerate(modValData.iterrows()):
        if len(nErrors) >= 5:
            for artistID in nErrors:
                print(f"del searchedForErrors['{artistID}']")
                stop=True
                break
        if any([dct.get(artistID) is not None for dct in [localArtistsDict, localArtistsDictErrors]]):
            continue
            
        artistName  = row["Name"]
        artistRef   = row["Ref"]

        print(f"{groupModVal: <8} |{j: <8} | {n: <8} | ", end="")
        try:
            response = webio.getArtistData(artistName=artistName, artistRef=artistRef)
        except:
            print("Error ==> {0}".format(artistName))
            localArtistsDictErrors[artistID] = True
            nErrors.append(artistID)
            webio.sleep(10)
            continue

        if not isinstance(response,bytes):
            print("Error ==> {0}".format(artistName))
            localArtistsDictErrors[artistID] = True
            nErrors.append(artistID)
            webio.sleep(3.5)
            continue

        nErrors = []
        modVal  = mv.getModVal(artistID)
        mio.data.saveRawArtistData(data=response, modval=modVal, dbID=artistID)
        localArtistsDict[artistID] = True
        webio.sleep(4.5)
        n += 1
        nLastErrors = 0
        
        if n % 5 == 0:
            if tt.isFinished():
                stop=True
                break

        if n % 50 == 0:
            webio.sleep(5)
            
        if n % 100 == 0:
            saveArtistData(db, localArtistsDict, localArtistsDictErrors)
            if tt.isFinished():
                stop=True
                break
            webio.wait(10.0)

        if n >= maxN:
            print("Breaking after {0} downloads...".format(maxN))
            stop=True
            break

ts.stop()
if True: saveArtistData(db, localArtistsDict, localArtistsDictErrors)

In [ ]:
if True: saveArtistData(db, localArtistsDict, localArtistsDictErrors)

# Download Album Data

## Create Media Data

In [ ]:
mediaData = {}
for modVal in range(100):
    modValData = mio.data.getModValData(modVal)
    modValMediaData = {}
    for artistID,artistIDData in modValData.iteritems():
        for mediaType,mediaTypeData in artistIDData.media.media.items():
            modValMediaData.update({code: [artistID,media.album,media.url] for code,media in mediaTypeData.items()})
    mediaData.update(modValMediaData)
    if (modVal+1) % 10 == 0:
        print(f"ModVal = {modVal+1}")
        
df = DataFrame(mediaData).T
df.columns = ["ArtistID", "Name", "Ref"]
knownMedia.save(data=df)

## Download Data

In [ ]:
mio   = bandcamp.MusicDBIO(verbose=False,local=True,mkDirs=False)
webio = bandcamp.RawWebData(debug=False)

In [ ]:
useArtist = False
numMaster = 250

knownAlbumsData = knownMedia.get()
knownAlbumsData['IndexModVal'] = knownAlbumsData.index.map(mio.getModVal)
availableNames  = concat([artistIDDF.head(numMaster) for artistID,artistIDDF in knownAlbumsData.groupby(["ArtistID"])])
localAlbumsDict = localAlbums.get()
availableNames  = availableNames[~availableNames.index.isin(localAlbumsDict.keys())]
albumNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby(["IndexModVal"])})

print(f"# {db} Album Search Results")
print(f"#   Available Album IDs:  {knownAlbumsData.shape[0]}")
print(f"#   Known Album IDs:      {len(localAlbumsDict)}")
print(f"#   Albums To Download:   {availableNames.shape[0]}")

del availableNames
del localAlbumsDict
del knownAlbumsData

#   Albums To Download:   23457
#   Albums To Download:   20437
#   Albums To Download:   12137
#   Albums To Download:   4132

In [ ]:
def saveAlbumData(db, localAlbumsDict, searchedForErrors):
    print("="*150)
    print(f"Saving {len(localAlbumsDict)} {db} Albums Data")
    localAlbums.save(data=localAlbumsDict)
    print(f"Saving {len(searchedForErrors)} {db} Searched For Errors")
    errors.save(data=searchedForErrors)
    print("="*150)

In [ ]:
ts = Timestat("Getting {0} AlbumIDs".format(db))
tt = getTT(skipEOD=False)

n    = 0
maxN = 25000000
localAlbumsDict     = localAlbums.get()
searchedForErrors   = errors.get()
stop = False
nErrors = []
for groupModVal,modValData in albumNamesToGet.iteritems():
    if stop is True:
        break
    for j,(albumID,row) in enumerate(modValData.iterrows()):
        if len(nErrors) >= 5:
            for artistID in nErrors:
                print(f"del searchedForErrors['{artistID}']")
                stop=True
                break
        if any([dct.get(albumID) is not None for dct in [localAlbumsDict, searchedForErrors]]):
            continue
            
        artistID   = row["ArtistID"]
        albumName  = row["Name"]
        albumRef  = row["Ref"]

        print(f"{groupModVal: <8} |{j: <8} | {n: <8} | ", end="")
        try:
            response = webio.getAlbumData(albumName=albumName, albumRef=albumRef)
        except:
            print("Error ==> {0}".format(albumName))
            searchedForErrors[albumID] = True
            nErrors.append(albumID)
            webio.sleep(10)
            continue

        if not isinstance(response,bytes):
            print("Error ==> {0}".format(albumName))
            searchedForErrors[albumID] = True
            nErrors.append(albumID)
            webio.sleep(3.5)
            continue

        nErrors = []
        modVal=mio.mv.get(albumID)
        mio.data.saveRawArtistAlbumData(data=response, modval=modVal, dbID=albumID)
        localAlbumsDict[albumID] = True
        webio.sleep(4.5)
        n += 1
        nLastErrors = 0
        
        if n % 5 == 0:
            if tt.isFinished():
                stop=True
                break

        if n % 50 == 0:
            webio.sleep(5)
            
        if n % 100 == 0:
            saveAlbumData(db, localAlbumsDict, searchedForErrors)
            if tt.isFinished():
                stop=True
                break
            webio.wait(10.0)

        if n >= maxN:
            print("Breaking after {0} downloads...".format(maxN))
            stop=True
            break

ts.stop()
if True: saveAlbumData(db, localAlbumsDict, searchedForErrors)

In [ ]:
from os import getpid
getpid()

# Download Lists

## Download Starter

In [ ]:
from apiutils import WebIO
from ioutils import FileIO, HTMLIO
io  = FileIO()
hio = HTMLIO()
wio = WebIO()

In [ ]:
starter             = {}
starter["List"]     = "https://www.bandcamp.org/lists.php"
starter["Genre"]    = "https://www.bandcamp.org/genre.php"
#starterBestAlbum = "https://www.bandcamp.org/ratings/6-highest-rated/2023/1"
starter["Rating"]   = "https://www.bandcamp.org/ratings"
starter["Discover"] = "https://www.bandcamp.org/discover"
starter["Releases"] = "https://www.bandcamp.org/releases"
starter["MustHear"] = "https://www.bandcamp.org/must-hear"

savename = "../../sandbox/AOTYstarter.p"
starterData = io.get(savename)
for key,url in starter.items():
    if starterData.get(key) is not None:
        continue
    retval = wio.get(url)
    if retval.code == 200:
        print(key)
        starterData[key] = retval.data
    wio.sleep(3)
        
print(f"Saving data to {savename}")
io.save(idata=starterData, ifile=savename)

In [ ]:
useStarter = False
useSite    = True

if useStarter is True:
    aotyData = io.get("../../sandbox/AOTYstarter.p")
elif useSite is True:
    aotyData = io.get("../../sandbox/AOTYsiteData.p") | io.get("../../sandbox/AOTYsiteData2.p")
else:
    aotyData = {}
    
N  = len(aotyData)
ts = Timestat(f"Sorting {N} Site Refs")
refsData = {"List": {}, "Lists": {}, "Rating": {}, "Album": {}, "Artist": {}, "Discover": {}, "Genre": {}, "MustHear": {}, "Release": {}, "Spotify": {}, "Apple": {}, "Amazon": {}, "User": {}}
for n,(key,keyData) in enumerate(aotyData.items()):
    if (n+1) % 500 == 0 or (n+1) == 100:
        ts.update(n=n+1, N=N)
        
    bsdata = hio.get(keyData)
    refs = bsdata.findAll("a")
    for ref in refs:
        href = ref.get('href')
        if not isinstance(href,str):
            continue
        if href.startswith("/list/"):
            refsData["List"][href] = ref.text.strip()
        elif "lists.php" in href:
            refsData["Lists"][href] = ref.text.strip()
        elif href.startswith("/ratings/"):
            refsData["Rating"][href] = ref.text.strip()
        elif href.startswith("/artist/"):
            refsData["Artist"][href] = ref.text.strip()
        elif href.startswith("/album/"):
            refsData["Album"][href] = ref.text.strip()
        elif href.startswith("/discover/"):
            refsData["Discover"][href] = ref.text.strip()
        elif href.startswith("/genre/"):
            refsData["Genre"][href] = ref.text.strip()
        elif href.startswith("/must-hear/"):
            refsData["MustHear"][href] = ref.text.strip()
        elif "/releases/" in href:
            refsData["Release"][href] = ref.text.strip()
        elif "spotify.com" in href:
            refsData["Spotify"][href] = ref.text.strip()
        elif "apple.com" in href:
            refsData["Apple"][href] = ref.text.strip()
        elif "amazon.com" in href:
            refsData["Amazon"][href] = ref.text.strip()
        elif "/user/" in href:
            refsData["User"][href] = ref.text.strip()
        else:
            continue
            print(href,'\t|\t',ref.text)
            
ts.stop()

In [ ]:
savename = "../../sandbox/AOTYsiteRefs.p"
print(f"Saving data to {savename}")
for key,keyData in refsData.items():
    print(f"  {key: <20}{len(keyData)}")
io.save(idata=refsData, ifile=savename)

In [ ]:
siteData   = {}
artistData = {}
albumData  = {}
userData   = {}
otherData  = {}

if False:
    for ref,name in refsData["List"].items():
        if ref.startswith("/artist/"):
            artistData[ref] = name
        elif ref.startswith("/album/"):
            albumData[ref] = name
        elif ref.startswith("/user/"):
            userData[ref] = name
        elif ref.startswith("/"):
            siteData[ref] = name
        else:
            otherData[ref] = name
else:
    for key,keyData in refsData.items():
        for ref,name in keyData.items():
            if ref.startswith("/artist/"):
                artistData[ref] = name
            elif ref.startswith("/album/"):
                albumData[ref] = name
            elif ref.startswith("/user/"):
                userData[ref] = name
            elif ref.startswith("/"):
                if "/list/" in ref:
                    siteData[ref] = name
                else:
                    otherData[ref] = name
            else:
                otherData[ref] = name
            
print(f"Found {len(artistData)} Artist Refs")
print(f"Found {len(albumData)} Album Refs")
print(f"Found {len(userData)} User Refs")
print(f"Found {len(siteData)} Site Refs")
print(f"Found {len(otherData)} Other Refs")

In [ ]:
for year in range(1970,2005):
    key = f'/lists.php?y={year}'
    val = 'View More'
    siteData[key] = val

In [ ]:
savename1 = "../../sandbox/AOTYsiteData.p"
siteDataDownloads1 = io.get(savename1)
print(f"Found {len(siteDataDownloads1)} Previous Downloads")
savename2 = "../../sandbox/AOTYsiteData2.p"
siteDataDownloads2 = io.get(savename2)
print(f"Found {len(siteDataDownloads2)} Previous Downloads")
N = len(siteData)
ts = Timestat(f"Downloading {N} Site Refs")
for n,(ref,name) in enumerate(siteData.items()):
    url=f"https://www.bandcamp.org{ref}"
    if any([dct.get(ref) is not None for dct in [siteDataDownloads1,siteDataDownloads2]]):
        continue
    
    retval = wio.get(url)
    if retval.code == 200:
        print(f"{n: <6} | {N: <6} | {ref}")
        siteDataDownloads2[ref] = retval.data
    wio.sleep(3)
    
    if (n+1) % 25 == 0:
        ts.update(n=n+1,N=N)
        print(f"Saving {len(siteDataDownloads2)} data to {savename2}")
        io.save(idata=siteDataDownloads2, ifile=savename2)

ts.stop()
        
print(f"Saving data to {savename2}")
io.save(idata=siteDataDownloads2, ifile=savename2)

In [ ]:
siteDataDownloads = io.get("../../sandbox/AOTYsiteData.p")

In [ ]:
io.save(idata=siteDataDownloads, ifile=savename)

## Check For New Data

In [ ]:
from lib.bandcamp import MusicDBID
mid = MusicDBID()
mid.getAlbumID('/album/515536-beyonce-renaissance/critic-lists/?f=all&y=2022')

In [ ]:
df = DataFrame(Series(refsData["Album"])).reset_index().rename(columns={"index": "Ref", 0: "List"})
df["AlbumID"] = df["Ref"].map(mid.getAlbumID)
df = df[~df["AlbumID"].duplicated()]

In [ ]:
df = DataFrame(Series(refsData["Artist"])).reset_index().rename(columns={"index": "Ref", 0: "Name"})
df["ArtistID"] = df["Ref"].map(mid.getArtistID)
df = df[~df["ArtistID"].duplicated()]

In [ ]:
artistNames = searchArtists()

In [ ]:
df.index = df["ArtistID"]
df = df.drop(["ArtistID"], axis=1)
artistNames = concat([artistNames,df])
artistNames = artistNames[~artistNames.index.duplicated()]

In [ ]:
mio.data.saveSearchArtistData(data=artistNames)

# Backup

In [ ]:
from utils import StoreData, backup
from numpy import array_split
sd = StoreData("bandcamp", "Artist")
for modVals in array_split(range(100), 2):
    sd.mergeLocalData(modVals=modVals)
sd.mergeGlobalData()

In [ ]:
from time import sleep
sleep(200)
sd.mergeGlobalData()

In [ ]:
for modVal in range(67):
    srcDir = DirInfo(f"/Volumes/Piggy/Discog/artists-bandcamp/{modVal}/artists")
    files  = [FileInfo(ifile) for ifile in srcDir.getFiles()]
    files  = [finfo for finfo in files if finfo.basename.isdigit()]
    dstDir = DirInfo(f"/Users/tgadfort/Music/Discog/artists-bandcamp/{modVal}/artists")
    for srcFile in files:
        dstFile = dstDir.join(srcFile.name)
        srcFile.mvFile(dstFile)